In [2]:
import torch 
import torchvision 
from torch.utils.data import  DataLoader,datasets,Subset
from torchvision import transforms, datasets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torchvision.models as models
from torch.autograd import Variable
from tqdm import tqdm,trange
#import torchtext
#from torchtext.data import get_tokenizer

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

# Main stuff

In [15]:
triplets_id = pd.read_csv('task4_be9ai3nsdj/train_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()
test_triplets_id = pd.read_csv('task4_be9ai3nsdj/test_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()

In [16]:
features = np.load('features_resnet18.npy')
features.shape

(10000, 512)

In [17]:
feature_transformed = features[triplets_id]#.reshape(59515, 512*3)
test_feature = features[test_triplets_id]
#feature_transformed = feature_transformed.transpose([0,2,1])

In [18]:
feature_transformed = torch.tensor(feature_transformed).to(device)
test_feature = torch.tensor(test_feature).to(device)
#features = torch.tensor(features)

In [72]:
feature_transformed.shape

torch.Size([59515, 3, 512])

In [19]:
print(feature_transformed.shape, test_feature.shape)

torch.Size([59515, 3, 512]) torch.Size([59544, 3, 512])


In [47]:
import torch.nn as nn
import torch.nn.functional as F

in_dim = feature_transformed.shape[2]
out_dim = 1

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(in_dim, 128) 
        self.hidden2 = nn.Linear(128, 64) 
        self.triplet_loss = nn.TripletMarginLoss(margin=0.5, p=2.0, reduction='none')
        
    def forward(self, x):
        x = F.logsigmoid(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.triplet_loss(x[:,0,:], x[:,1,:], x[:,2,:])
        
        return x

net = Net()

In [48]:
net.to(device)

Net(
  (hidden1): Linear(in_features=512, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=64, bias=True)
  (triplet_loss): TripletMarginLoss()
)

In [49]:
forward = net(feature_transformed)
forward

tensor([0.8514, 0.4409, 0.4658,  ..., 0.3276, 0.5653, 0.4281], device='cuda:0',
       grad_fn=<ClampMinBackward>)

In [86]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=1e-4)

In [87]:
num_epochs = 20

In [88]:
%%time
pbar = trange(num_epochs)
for i in pbar:  # loop over the dataset multiple times

    running_loss = 0.0
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(feature_transformed)
    
    criterion = torch.mean(outputs)
    criterion.backward()
    
    optimizer.step()
    
#     print(criterion)
    pbar.set_description(f"mean {criterion}")

mean 0.04395558685064316: 100%|██████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.99it/s]

Wall time: 5 s


In [64]:
torch.sum(net(feature_transformed)<0.5)

tensor(55838, device='cuda:0')

In [66]:
prediction = (net(test_feature)<0.5).cpu()

In [67]:
prediction = np.array(prediction, dtype=int)

In [68]:
np.shape(prediction)

(59544,)

In [70]:
pd.DataFrame(prediction.T).to_csv('prediction.csv', index=False, header=None)